# LangChain RAG with Locall LLM Tutorial

This is based on Pixegami's tutorial. ([original repo](https://github.com/pixegami/rag-tutorial-v2/))

## Loading The Data

More on Document loaders --> [official document](https://python.langchain.com/v0.1/docs/modules/data_connection/document_loaders/).

In [ ]:
!git clone https://github.com/pixegami/rag-tutorial-v2/
!mv rag-tutorial-v2/data ./
!rm -rf rag-tutorial-v2

In [1]:
from langchain.document_loaders.pdf import PyPDFDirectoryLoader

DATA_PATH = "data"

def load_documents():
    document_loader = PyPDFDirectoryLoader(DATA_PATH)
    return document_loader.load()

In [2]:
documents = load_documents()
print(len(documents))
print(documents[0])

12
page_content='On a blustery autumn evening five old friends met in the backroom of one of the city’s oldest and most private clubs. Each had\ntraveled a long distance — from all corners of the world — to meet on this very specific day… October 2, 1900 — 28 years to the\nday that the London eccentric, Phileas Fogg accepted and then won a £20,000 bet that he could travel Around the World in 80 Days . \nWhen the story of Fogg’s triumphant journey filled all the newspapers of the day, the five attended University together. Inspired by\nhis impetuous gamble, and a few pints from the local pub, the group commemorated his circumnavigation with a more modest excur-sion and wager – a bottle of good claret to the first to make it to Le Procope in Paris.\nEach succeeding year, they met to celebrate the anniversary and pay tribute to Fogg. And each year a new expedition (always mor e\ndifficult) with a new wager (always more expensive) was proposed. Now at the dawn of the century it was time fo

## Split The Documents 

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema.document import Document

def split_documents(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=80,
        length_function=len,
        is_separator_regex=False,
    )
    return text_splitter.split_documents(documents)

In [4]:
chunks = split_documents(documents)
print(len(chunks))
print(chunks[0])

41
page_content='On a blustery autumn evening five old friends met in the backroom of one of the city’s oldest and most private clubs. Each had\ntraveled a long distance — from all corners of the world — to meet on this very specific day… October 2, 1900 — 28 years to the\nday that the London eccentric, Phileas Fogg accepted and then won a £20,000 bet that he could travel Around the World in 80 Days . \nWhen the story of Fogg’s triumphant journey filled all the newspapers of the day, the five attended University together. Inspired by\nhis impetuous gamble, and a few pints from the local pub, the group commemorated his circumnavigation with a more modest excur-sion and wager – a bottle of good claret to the first to make it to Le Procope in Paris.' metadata={'source': 'data/ticket_to_ride.pdf', 'page': 0}


## Embedding Function

### Using Cloud Embedding Function

Cloud hosted embedding function generally performs better.

In [ ]:
from langchain_community.embeddings.bedrock import BedrockEmbeddings

def get_embedding_function():
    embeddings = BedrockEmbeddings(
        credentials_profile_name="default", region_name="us-east-1"
    )
    # embeddings = OllamaEmbeddings(model="nomic-embed-text")
    return embeddings

### Running **Local** Embedding Function

In case you want to run the embedding function locally as well, run the following cell instead.

In [ ]:
!ollama list

In [ ]:
!ollama pull nomic-embed-text

In [5]:
from langchain_community.embeddings.ollama import OllamaEmbeddings

def get_embedding_function():
    embeddings = OllamaEmbeddings(model="nomic-embed-text")
    return embeddings

## Creating The Database

In [6]:
from langchain.vectorstores.chroma import Chroma

CHROMA_PATH = "chroma"

def add_to_chroma(chunks: list[Document]):
    # Load the existing database.
    db = Chroma(
        persist_directory=CHROMA_PATH, embedding_function=get_embedding_function()
    )

    # Calculate Page IDs.
    chunks_with_ids = calculate_chunk_ids(chunks)

    # Add or Update the documents.
    existing_items = db.get(include=[])  # IDs are always included by default
    existing_ids = set(existing_items["ids"])
    print(f"Number of existing documents in DB: {len(existing_ids)}")

    # Only add documents that don't exist in the DB.
    new_chunks = []
    for chunk in chunks_with_ids:
        if chunk.metadata["id"] not in existing_ids:
            new_chunks.append(chunk)

    if len(new_chunks):
        print(f"👉 Adding new documents: {len(new_chunks)}")
        new_chunk_ids = [chunk.metadata["id"] for chunk in new_chunks]
        db.add_documents(new_chunks, ids=new_chunk_ids)
        db.persist()
    else:
        print("✅ No new documents to add")
        

def calculate_chunk_ids(chunks):

    # This will create IDs like "data/monopoly.pdf:6:2"
    # Page Source : Page Number : Chunk Index

    last_page_id = None
    current_chunk_index = 0

    for chunk in chunks:
        source = chunk.metadata.get("source")
        page = chunk.metadata.get("page")
        current_page_id = f"{source}:{page}"

        # If the page ID is the same as the last one, increment the index.
        if current_page_id == last_page_id:
            current_chunk_index += 1
        else:
            current_chunk_index = 0

        # Calculate the chunk ID.
        chunk_id = f"{current_page_id}:{current_chunk_index}"
        last_page_id = current_page_id

        # Add it to the page meta-data.
        chunk.metadata["id"] = chunk_id

    return chunks

In [7]:
add_to_chroma(chunks)

Number of existing documents in DB: 41
✅ No new documents to add


In [8]:
!du -ah ./chroma

4.0K	./chroma/ef219065-076e-496e-be5b-b3fa0deb744a/length.bin
4.0K	./chroma/ef219065-076e-496e-be5b-b3fa0deb744a/header.bin
3.1M	./chroma/ef219065-076e-496e-be5b-b3fa0deb744a/data_level0.bin
0	./chroma/ef219065-076e-496e-be5b-b3fa0deb744a/link_lists.bin
3.1M	./chroma/ef219065-076e-496e-be5b-b3fa0deb744a
552K	./chroma/chroma.sqlite3
3.7M	./chroma


## Running RAG Query Locally

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_community.llms.ollama import Ollama

CHROMA_PATH = "chroma"

PROMPT_TEMPLATE = """
Answer the question based only on the following context:
{context}

---
Answer the question based on the above context: {question}
"""

In [ ]:
embedding_function = get_embedding_function()
db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

In [ ]:
query_text="How many clues can I give in Codenames?"

In [ ]:
# Search the DB.
results = db.similarity_search_with_score(query_text, k=5)

In [ ]:
context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, question=query_text)
print(prompt)

In [ ]:
model = Ollama(model="mistral")

In [ ]:
response_text = model.invoke(prompt)

In [ ]:
print(response_text)

In [ ]:
sources = [doc.metadata.get("id", None) for doc, _score in results]
formatted_response = f"Response: {response_text}\nSources: {sources}"
print(formatted_response)

### Function-ized

In [9]:
from langchain.prompts import ChatPromptTemplate
from langchain_community.llms.ollama import Ollama

CHROMA_PATH = "chroma"

PROMPT_TEMPLATE = """
Answer the question based only on the following context:
{context}

---
Answer the question based on the above context: {question}
"""

def query_rag(query_text: str):
    # Prepare the DB.
    embedding_function = get_embedding_function()
    db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

    # Search the DB.
    results = db.similarity_search_with_score(query_text, k=5)

    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query_text)
    # print(prompt)

    model = Ollama(model="mistral")
    response_text = model.invoke(prompt)

    sources = [doc.metadata.get("id", None) for doc, _score in results]
    formatted_response = f"Response: {response_text}\nSources: {sources}"
    print(formatted_response)
    return response_text

In [12]:
query_text="How do I get out of jail in Monopoly?"

In [13]:
query_rag(query_text)

Response:  To get out of jail in Monopoly, you have several options: (1) throw doubles on any of your next three turns, (2) use a "Get Out of Jail Free" card if you have one, (3) purchase a "Get Out of Jail Free" card from another player and play it, or (4) pay a fine of $50 before rolling the dice on either of your next two turns. If you do not throw doubles by your third turn, you must pay the $50 fine and move forward the number of spaces shown by your next roll. You can also buy and sell property, buy and sell houses and hotels, and collect rents while in jail. However, you cannot collect your $200 salary if sent to jail during a move.
Sources: ['data/monopoly.pdf:1:0', 'data/monopoly.pdf:0:0', 'data/monopoly.pdf:4:1', 'data/monopoly.pdf:1:1', 'data/monopoly.pdf:4:2']


' To get out of jail in Monopoly, you have several options: (1) throw doubles on any of your next three turns, (2) use a "Get Out of Jail Free" card if you have one, (3) purchase a "Get Out of Jail Free" card from another player and play it, or (4) pay a fine of $50 before rolling the dice on either of your next two turns. If you do not throw doubles by your third turn, you must pay the $50 fine and move forward the number of spaces shown by your next roll. You can also buy and sell property, buy and sell houses and hotels, and collect rents while in jail. However, you cannot collect your $200 salary if sent to jail during a move.'

In [ ]:
query_text="How many clues can I give in Codenames?"